In [1]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 1
HIDDEN_LAYER_SIZE = 256
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 200


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = nn.SingleLayerRNN(name = "multi-rakim")
rnn.build(NUM_LAYERS, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0, embedding=embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1, embedding=True)

(1824, 200)
(?, 20)


Epoch 1/20
Loss:    	 7.508976936340332
Accuracy:	 0.01032204832881689
yeah you know what this is nyc
the triumphant return rakim allah
rakim
remember being introduced to
Seed:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to 
Result:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to lap prepare taking graffiti formula work rush comedian show stolen walking high flows mixing which closer busting fahrenheit live only friday originator recite shit mens borroughs worldwide except killing 'cause died part nah counter down emceeing tempo became lost omega pass relax thick kinds ghettos momentum east respond dj's gettin family no president's slash stereo constant sounds stream quiet 48 if sense enemy corner measured killed torture impacted past became solve torture loud since stream chewin no crime forget prepare hyper door intestines glory started favorite bud

In [1]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)

NUM_LAYERS = 1
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 50

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 150


data, labels = vocab.making_embedded_one_hot(text, TIMESTEPS)

embedding = nn.LeanableEmbedding(name = "learnable-embedding")
embedding.build(VOCAB_SIZE, EMBEDDING_SIZE)

rnn = nn.SingleLayerRNN(name = "single-rakim-100")
rnn.build(NUM_LAYERS, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0, embedding=embedding)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1, embedding=True)

(1824, 150)
(?, 50)


Epoch 1/20
Loss:    	 7.508759021759033
Accuracy:	 0.031062331050634384
yeah you know what this is nyc
the triumphant return rakim allah
rakim
remember being introduced to rapping your first rhyme
it's pivotal like a fiends first high
hustlers first ride players first dime
unforgettable like a ladies first time
the world been
Seed:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to rapping your first rhyme 
 it's pivotal like a fiends first high 
 hustlers first ride players first dime 
 unforgettable like a ladies first time 
 the world been 
Result:yeah you know what this is nyc 
 the triumphant return rakim allah 
 rakim 
 remember being introduced to rapping your first rhyme 
 it's pivotal like a fiends first high 
 hustlers first ride players first dime 
 unforgettable like a ladies first time 
 the world been fiends eager 1st last shorty hood figured delicatessen intelligent always stereo scared navigatin